In [14]:
# Importing libraries
import os
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import streamlit as st
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [3]:
# Setting Streamlit UI
st.title("Welcome to DataWhiz!")
st.subheader("Conversations With Data, Made Simple")

2025-02-22 15:31:32.764 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 15:31:34.237 
  command:

    streamlit run C:\Users\prath\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-22 15:31:34.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 15:31:34.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 15:31:34.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [17]:
# Read the CSV dataset to be converted into dataframe.
csv_path = "data/titanic.csv"
df = pd.read_csv(csv_path)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [26]:
df[df["Sex"]=="male"].Survived==1

0      False
4      False
5      False
6      False
7      False
       ...  
883    False
884    False
886    False
889     True
890    False
Name: Survived, Length: 577, dtype: bool

In [5]:
# Setting OpenAI model.
model = "gpt-4o-mini"

# Setting OpenAI API key.
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [27]:
# Setting the LLM.
llm = ChatOpenAI(model=model, temperature=0)

# Setting the Agent type.
agent_type = AgentType.OPENAI_FUNCTIONS

st.session_state.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Setting the Agent to interact with Dataframe.
df_agent = create_pandas_dataframe_agent(
    llm=llm, 
    df=df, 
    agent_type=agent_type,
    suffix="Also return a JSON dictionary that can be parsed into a dataframe containing the requested information",
    verbose=True,
    allow_dangerous_code=True,
    memory=st.session_state.memory)

2025-02-22 18:13:59.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 18:13:59.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 18:13:59.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\prath\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'memory': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')} which are no longer supported.
  warnings.warn(


In [35]:
df_agent.invoke("Based on the correlation values with survived column, what other columns are most correlated with it?")




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "correlation_matrix = df.corr()\ncorrelation_with_survived = correlation_matrix['survived'].sort_values(ascending=False)\ncorrelation_with_survived"}`


ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'
Invoking: `python_repl_ast` with `{'query': "# First, let's check the data types of the columns to identify non-numeric columns\ncolumn_types = df.dtypes\ncolumn_types"}`


PassengerId       int64
Survived          int64
Pclass         category
Name             object
Sex            category
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked         object
dtype: object
Invoking: `python_repl_ast` with `{'query': "# We need to convert categorical columns to numeric for correlation analysis\n# Let's convert 'Pclass', 'Sex', and 'Embarked' to numeric values\ndf['Pclass'] = df['Pclas

{'input': 'Based on the correlation values with survived column, what other columns are most correlated with it?',
 'output': 'The correlation values with the "Survived" column are as follows:\n\n- **Survived**: 1.000000 (self-correlation)\n- **Fare**: 0.257307\n- **Parch**: 0.081629\n- **PassengerId**: -0.005007\n- **SibSp**: -0.035322\n- **Age**: -0.077221\n- **Sex**: -0.543351\n\nFrom this, we can see that the columns most correlated with "Survived" are "Fare" and "Sex".\n\nHere is the JSON dictionary containing the correlation values:\n\n```json\n{\n    "Survived": 1.000000,\n    "Fare": 0.257307,\n    "Parch": 0.081629,\n    "PassengerId": -0.005007,\n    "SibSp": -0.035322,\n    "Age": -0.077221,\n    "Sex": -0.543351\n}\n```'}